In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from matplotlib.colors import ListedColormap

In [21]:
# --- Parameters ---
original_root = "/Users/ereilly/Documents/code/autoSAR_preprocessing/test/output"
mask_root = "/Users/ereilly/Documents/code/autoSAR_preprocessing/output_postprocessing"
output_plot_dir = "/Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images"

In [22]:
overlay_cmap = ListedColormap([
    (0.0, 0.0, 0.0, 0.0),   # transparent
    (1.0, 0.5, 0.0, 1.0)    # solid orange
])


In [23]:
# --- Function to get original image path based on naming pattern ---
def match_original_path(mask_filename):
    return os.path.join(original_root, mask_filename.replace("_JPL0.4_VVDR_cumulative_mask.tif", ".tif"))

In [29]:
# Plot side-by-side: original + overlay
def plot_overlay(original_path, mask_path, output_path):
    with rasterio.open(original_path) as src:
        original = src.read(1)
        bounds = src.bounds
        extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

    with rasterio.open(mask_path) as src:
        mask = src.read(1)

    # Normalize original using standard deviation
    mean = np.nanmean(original)
    std = np.nanstd(original)
    z_scaled = (original - mean) / std
    clipped = np.clip(z_scaled, -2, 2)
    normed = (clipped + 2) / 4  # range [0, 1]

    # Plot side-by-side
    fig, axes = plt.subplots(1, 2, figsize=(10, 12), constrained_layout=True)


    # Left: original
    axes[0].imshow(normed, cmap="gray", extent=extent)
    axes[0].set_title("Pre-Processed", fontsize=16)
    axes[0].axis('off')

    # Right: original + mask
    axes[1].imshow(normed, cmap="gray", extent=extent)
    axes[1].imshow(mask, cmap=overlay_cmap, extent=extent)
    axes[1].set_title("Output with Mask", fontsize=16)
    axes[1].axis('off')

    os.makedirs(output_plot_dir, exist_ok=True)
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {output_path}")

In [30]:
# --- Iterate over all masks ---
for root, _, files in os.walk(mask_root):
    for file in files:
        if file.endswith("_JPL0.4_VVDR_cumulative_mask.tif"):
            mask_path = os.path.join(root, file)
            original_path = match_original_path(file)

            if os.path.exists(original_path):
                # Save all to flat folder, name derived from base name
                out_name = file.replace("_JPL0.4_VVDR_cumulative_mask.tif", "_overlay.png")
                output_path = os.path.join(output_plot_dir, out_name)
                plot_overlay(original_path, mask_path, output_path)
            else:
                print(f"[!] Original not found for: {file}")

Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/S1A_IW_GRDH_1SDV_20241216T015014_20241216T015043_057011_070161_E8F2_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/S1A_IW_GRDH_1SDV_20241129T134455_20241129T134520_056770_06F7DB_D673_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/S1A_IW_GRDH_1SDV_20241110T135319_20241110T135344_056493_06ECD4_BBBF_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/S1A_IW_GRDH_1SDV_20241017T015017_20241017T015046_056136_06DE9F_4EAA_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/S1A_IW_GRDH_1SDV_20240830T015015_20240830T015045_055436_06C312_73FC_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/S1A_IW_GRDH_1SDV_20241029T015017_20241029T015046_056311_06E595_EEAE_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preproc

In [39]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from datetime import datetime
from pytz import timezone, utc
from matplotlib.colors import ListedColormap
from pytz import timezone


In [40]:
# --- Parameters ---
original_root = "/Users/ereilly/Documents/code/autoSAR_preprocessing/test/output"
mask_root = "/Users/ereilly/Documents/code/autoSAR_preprocessing/output_postprocessing"
output_plot_dir = "/Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/wind"
met_csv_path = "/Volumes/External/TJ_estuary/analysis/TJRTLMET.csv"


In [47]:

pacific = timezone("US/Pacific")
# --- Load meteorological data (in Pacific Time) ---
met_df = pd.read_csv(met_csv_path)
met_df.columns = met_df.columns.str.strip()

met_df["DateTimeStamp"] = pd.to_datetime(met_df["DateTimeStamp"], format="%m/%d/%y %H:%M")
met_df["DateTimeStamp"] = met_df["DateTimeStamp"].dt.tz_localize(pacific, ambiguous='NaT')
met_df = met_df.dropna(subset=["DateTimeStamp"])
met_df = met_df.sort_values("DateTimeStamp")

# --- Overlay color map: solid orange mask ---
overlay_cmap = ListedColormap([
    (0.0, 0.0, 0.0, 0.0),   # transparent
    (1.0, 0.5, 0.0, 1.0)    # solid orange
])


In [63]:
def extract_datetime_from_filename(filename):
    """Extract UTC datetime from Sentinel-1 filename."""
    parts = filename.split("_")
    for part in parts:
        if part.startswith("20") and "T" in part:
            try:
                return utc.localize(datetime.strptime(part, "%Y%m%dT%H%M%S"))
            except ValueError:
                continue
    return None

def get_nearest_met_data(local_time):
    """Find the closest met record to the given local time."""
    time_deltas = (met_df["DateTimeStamp"] - local_time).abs()
    nearest_idx = time_deltas.idxmin()
    row = met_df.loc[nearest_idx]
    return row["WSpd"], row["Wdir"], row["DateTimeStamp"]

def match_original_path(mask_filename):
    return os.path.join(original_root, mask_filename.replace("_JPL0.4_VVDR_cumulative_mask.tif", ".tif"))

def plot_overlay(original_path, mask_path, output_path):
    # --- Extract and convert image time ---
    image_time_utc = extract_datetime_from_filename(os.path.basename(original_path))
    image_time_local = image_time_utc.astimezone(pacific)

    # --- Match meteorological data ---
    wspd, wdir, met_time = get_nearest_met_data(image_time_local)

    # --- Read images ---
    with rasterio.open(original_path) as src:
        original = src.read(1)
        bounds = src.bounds
        extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

    with rasterio.open(mask_path) as src:
        mask = src.read(1)

    # --- Normalize image using standard deviation scaling ---
    mean = np.nanmean(original)
    std = np.nanstd(original)
    z_scaled = (original - mean) / std
    clipped = np.clip(z_scaled, -2, 2)
    normed = (clipped + 2) / 4  # Scale to [0, 1]

    # --- Set up side-by-side figure ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 12), constrained_layout=True)

    # Left: original only
    axes[0].imshow(normed, cmap="gray", extent=extent)
    axes[0].set_title("Original (Normalized)", fontsize=12)
    axes[0].axis('off')

    # Right: original + mask
    axes[1].imshow(normed, cmap="gray", extent=extent)
    axes[1].imshow(mask, cmap=overlay_cmap, extent=extent)
    axes[1].set_title("Original with Mask", fontsize=12)
    axes[1].axis('off')

    # --- Bottom annotation: wind info + local time ---
    wind_text = (
        f"Wind: {wspd:.1f} m/s @ {wdir:.0f}°\n"
        f"Local Time: {image_time_local.strftime('%Y-%m-%d %H:%M')}"
    )
    fig.text(0.5, 0.015, wind_text, ha='center', fontsize=13, fontweight='bold')


    # --- Wind vector in real coordinates ---
    import matplotlib.patches as patches
    import math


    # --- Compute arrow parameters ---
    wind_angle_deg = (270 - wdir) % 360  # convert "from" to math angle
    angle_rad = math.radians(wind_angle_deg)

    # Get image height (in spatial units)
    height = bounds.top - bounds.bottom

    # Base arrow length as % of image height, scaled by wind speed
    base_frac = 0.07  # base length = 7% of image height
    scale = np.clip(wspd / 10, 0.5, 1.5)  # scale between 50%–150%
    arrow_length = height * base_frac * scale

    # Arrow vector
    dx = arrow_length * math.cos(angle_rad)
    dy = arrow_length * math.sin(angle_rad)

    # Starting point (near bottom-right of image)
    x0 = bounds.right - 0.1 * (bounds.right - bounds.left)
    y0 = bounds.bottom + 0.05 * height

    # --- Draw the arrow ---
    axes[1].add_patch(
        patches.FancyArrow(
            x0, y0, dx, dy,
            width=arrow_length * 0.05,
            head_width=arrow_length * 0.15,
            head_length=arrow_length * 0.15,
            length_includes_head=True,
            transform=axes[1].transData,
            color='lime',
            alpha=0.9
        )
    )

    # --- Save plot ---
    os.makedirs(output_plot_dir, exist_ok=True)
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved: {output_path}")



In [64]:
# --- Main processing loop ---
for root, _, files in os.walk(mask_root):
    for file in files:
        if file.endswith("_JPL0.4_VVDR_cumulative_mask.tif"):
            mask_path = os.path.join(root, file)
            original_path = match_original_path(file)

            if os.path.exists(original_path):
                flat_name = file.replace("_JPL0.4_VVDR_cumulative_mask.tif", "_overlay.png")
                output_path = os.path.join(output_plot_dir, flat_name)
                plot_overlay(original_path, mask_path, output_path)
            else:
                print(f"[!] Original image not found for: {file}")


Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/wind/S1A_IW_GRDH_1SDV_20241216T015014_20241216T015043_057011_070161_E8F2_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/wind/S1A_IW_GRDH_1SDV_20241129T134455_20241129T134520_056770_06F7DB_D673_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/wind/S1A_IW_GRDH_1SDV_20241110T135319_20241110T135344_056493_06ECD4_BBBF_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/wind/S1A_IW_GRDH_1SDV_20241017T015017_20241017T015046_056136_06DE9F_4EAA_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/wind/S1A_IW_GRDH_1SDV_20240830T015015_20240830T015045_055436_06C312_73FC_pre_overlay.png
Saved: /Users/ereilly/Documents/code/autoSAR_preprocessing/plotted_images/wind/S1A_IW_GRDH_1SDV_20241029T015017_20241029T015046_056311_06E595_EEAE_pre_overlay.png
Saved: /Users/ereilly/